In [ ]:
!pip install langchain_community
!pip install pypdfium2
!pip install langchain_experimental

In [ ]:
!pip install faiss-gpu-cu12

In [ ]:
!pip install rank_bm25

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import re
from tqdm import tqdm
from langchain_community.document_loaders import PyPDFium2Loader
from langchain_experimental.text_splitter import SemanticChunker
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

def preprocessing_pdf(text):
    """특정 페이지 전체 및 불필요한 텍스트 제거"""
    text = re.sub(r'KOSHA GUIDE', '', text)
    text = re.sub(r'^C - .+$', '', text, flags=re.MULTILINE)
    text = re.sub(r'^<그림\s*\d+\s*>$', '', text, flags=re.MULTILINE)
    text = re.sub(r'^\s*-\s*\d+\s*-\s*$', '', text, flags=re.MULTILINE)
    return text.strip()

# PDF 파일들이 있는 폴더 경로 설정
pdf_dir = "/content/drive/MyDrive/PDF"
pdf_files = [os.path.join(pdf_dir, f) for f in os.listdir(pdf_dir) if f.lower().endswith('.pdf')]

embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
text_splitter = SemanticChunker(embeddings)

all_chunks = []

# tqdm을 사용하여 PDF 파일 처리 진행 상황 추적
for pdf_file in tqdm(pdf_files, desc="Processing PDFs"):
    print(f"\nProcessing: {pdf_file}")
    loader = PyPDFium2Loader(pdf_file)
    documents = loader.load()
    # 3페이지(인덱스 2)부터 전처리한 텍스트 추출
    page_contents = []
    for i, doc in enumerate(documents[2:], start=3):
        processed_text = preprocessing_pdf(doc.page_content)
        page_contents.append(processed_text)
        print(f"  Processed page {i}")
    # 문서별로 페이지 텍스트들을 합치기
    doc_text = "\n".join(page_contents)
    # 문서별로 청크 생성
    chunks = text_splitter.split_text(doc_text)
    print(f"  Generated {len(chunks)} chunks")
    all_chunks.extend(chunks)

vectorstore = FAISS.from_texts(all_chunks, embeddings)
vectorstore.save_local("faiss_index")
print("FAISS index saved successfully.")


In [ ]:
vectorstore

In [ ]:
# bm25 토크나이저로 한국어 토크나이저 사용 하기 위해서 불러옴
from kiwipiepy import Kiwi
kiwi = Kiwi()

def ko_kiwi_tokenizer(text: str):
    # Kiwi 토크나이저는 각 토큰에 대한 다양한 정보를 반환합니다.
    # 여기서는 토큰의 표면 형태(텍스트)만 추출합니다.
    tokens = kiwi.tokenize(text)
    return [token[0] for token in tokens]

In [ ]:
# Retriever 정의
from langchain_community.vectorstores import FAISS
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.docstore.document import Document

docs = [Document(page_content=doc) for doc in all_chunks]
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})
bm25_retriever = BM25Retriever.from_documents(docs, tokenizer=ko_kiwi_tokenizer)


In [ ]:
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, retriever],
    weights=[0.0, 1.0]  # 각 리트리버에 동일 가중치 부여 (가중치 합은 1.0)
)

In [ ]:
# 사용자 질의 예시
query = "건축물' 공사 중 철근콘크리트공사' 작업에서  '타설작업 작업프로세스 진행중 부딪힘 발생 했습니다 사고 원인은 '펌프카 아웃트리거 바닥 고임목을 3단으로 보강 했음에도, 지반 침하(아웃트리거 우측 상부 1개소)가 발생하였고,  좌, 우측 아웃트리거의 펼친 길이가 상이하고 타설 위치가 건물 끝부분 모서리에 위치하여 붐대호스를 최대로 펼치다 보니 장비에 대한 무게중심이 한쪽으로 쏠려 일부 전도되는 사고가 발생된 것으로 판단됨'입니다.재발 방지 대책 및 향후 조치 계획은 무엇인가요?"

# ensemble_retriever를 이용해 관련 문서 검색 (두 리트리버의 결과를 결합하여 가중치에 따라 정렬)
retrieved_docs = ensemble_retriever.get_relevant_documents(query)

# 검색된 문서 출력
print("검색된 문서들:")
for idx, doc in enumerate(retrieved_docs):
    print(f"\n문서 {idx+1}:")
    print(doc.page_content)